# 第9章 时序数据

In [1]:
import pandas as pd
import numpy as np

## 一、时序的创建
###  1. 四类时间变量
#### 现在理解可能关于③和④有些困惑，后面会作出一些说明

名称 | 描述 | 元素类型 | 创建方式  
:-|:-|:-|:-
① Date times（时间点） | 描述特定日期或时间点 | Timestamp | to_datetime或date_range
② Time spans（时期） | 由时间点定义的一段时期 | Period | Period或period_range
③ Date offsets（相对时间差） | 一段时间的相对大小（与夏/东令时无关） | DateOffset | DateOffset
④ Time deltas（绝对时间差） | 一段时间的绝对大小（与夏/东令时有关） | Timedelta | to_timedelta或timedelta_range

### 2. 时间点的创建

#### （a）to_datetime方法
#### Pandas在时间点建立的输入格式规定上给了很大的自由度，下面的语句都能正确建立同一时间点

In [2]:
pd.to_datetime('2020.1.1')
pd.to_datetime('2020 1.1')
pd.to_datetime('2020 1 1')
pd.to_datetime('2020 1-1')
pd.to_datetime('2020-1 1')
pd.to_datetime('2020-1-1')
pd.to_datetime('2020/1/1')
pd.to_datetime('1.1.2020')
pd.to_datetime('1.1 2020')
pd.to_datetime('1 1 2020')
pd.to_datetime('1 1-2020')
pd.to_datetime('1-1 2020')
pd.to_datetime('1-1-2020')
pd.to_datetime('1/1/2020')
pd.to_datetime('20200101')
pd.to_datetime('2020.0101')

Timestamp('2020-01-01 00:00:00')

#### 下面的语句都会报错

In [3]:
#pd.to_datetime('2020\\1\\1')
#pd.to_datetime('2020`1`1')
#pd.to_datetime('2020.1 1')
#pd.to_datetime('1 1.2020')

#### 此时可利用format参数强制匹配

In [4]:
pd.to_datetime('2020\\1\\1',format='%Y\\%m\\%d')
pd.to_datetime('2020`1`1',format='%Y`%m`%d')
pd.to_datetime('2020.1 1',format='%Y.%m %d')
pd.to_datetime('1 1.2020',format='%d %m.%Y')

Timestamp('2020-01-01 00:00:00')

#### 同时，使用列表可以将其转为时间点索引

In [5]:
pd.Series(range(2),index=pd.to_datetime(['2020/1/1','2020/1/2']))

2020-01-01    0
2020-01-02    1
dtype: int64

In [6]:
type(pd.to_datetime(['2020/1/1','2020/1/2']))

pandas.core.indexes.datetimes.DatetimeIndex

#### 对于DataFrame而言，如果列已经按照时间顺序排好，则利用to_datetime可自动转换

In [7]:
df = pd.DataFrame({'year': [2020, 2020],'month': [1, 1], 'day': [1, 2]})
pd.to_datetime(df)

0   2020-01-01
1   2020-01-02
dtype: datetime64[ns]

#### （b）时间精度与范围限制
#### 事实上，Timestamp的精度远远不止day，可以最小到纳秒ns

In [8]:
pd.to_datetime('2020/1/1 00:00:00.123456789')

Timestamp('2020-01-01 00:00:00.123456789')

#### 同时，它带来范围的代价就是只有大约584年的时间点是可用的

In [9]:
pd.Timestamp.min

Timestamp('1677-09-21 00:12:43.145225')

In [10]:
pd.Timestamp.max

Timestamp('2262-04-11 23:47:16.854775807')

#### （c）date_range方法
#### 一般来说，start/end/periods（时间点个数）/freq（间隔方法）是该方法最重要的参数，给定了其中的3个，剩下的一个就会被确定

In [11]:
pd.date_range(start='2020/1/1',end='2020/1/10',periods=3)

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-05 12:00:00',
               '2020-01-10 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [12]:
pd.date_range(start='2020/1/1',end='2020/1/10',freq='D')

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
               '2020-01-09', '2020-01-10'],
              dtype='datetime64[ns]', freq='D')

In [13]:
pd.date_range(start='2020/1/1',periods=3,freq='D')

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03'], dtype='datetime64[ns]', freq='D')

In [14]:
pd.date_range(end='2020/1/3',periods=3,freq='D')

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03'], dtype='datetime64[ns]', freq='D')

#### 其中freq参数有许多选项，下面将常用部分罗列如下

符号 | D/B | W | M/Q/Y | BM/BQ/BY | MS/QS/YS | BMS/BQS/BYS | H | T | S
:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
描述 | 日/工作日 | 周 | 月末 | 月/季/年末日 | 月/季/年末工作日 | 月/季/年初日 | 月/季/年初工作日 | 小时 | 分钟 |秒

In [15]:
pd.date_range(start='2020/1/1',periods=3,freq='T')

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-01 00:01:00',
               '2020-01-01 00:02:00'],
              dtype='datetime64[ns]', freq='T')

In [16]:
pd.date_range(start='2020/1/1',periods=3,freq='M')

DatetimeIndex(['2020-01-31', '2020-02-29', '2020-03-31'], dtype='datetime64[ns]', freq='M')

In [17]:
pd.date_range(start='2020/1/1',periods=3,freq='BYS')

DatetimeIndex(['2020-01-01', '2021-01-01', '2022-01-03'], dtype='datetime64[ns]', freq='BAS-JAN')

#### bdate_range是一个类似与date_range的方法，特点在于可以在自带的工作日间隔设置上，再选择weekmask参数和holidays参数
#### 它的freq中有一个特殊的'C'/'CBM'/'CBMS'选项，表示定制，需要联合weekmask参数和holidays参数使用
#### 例如现在需要将周一、周二、周五3天保留，并将部分holidays剔除

In [18]:
weekmask = 'Mon Tue Fri'
holidays = [pd.Timestamp('2020/1/%s'%i) for i in range(7,13)]
#注意holidays
pd.bdate_range(start='2020-1-1',end='2020-1-15',freq='C',weekmask=weekmask,holidays=holidays)

DatetimeIndex(['2020-01-03', '2020-01-06', '2020-01-13', '2020-01-14'], dtype='datetime64[ns]', freq='C')

### 3. DateOffset对象

#### （a）DataOffset与Timedelta的区别
#### Timedelta绝对时间差的特点指无论是冬令时还是夏令时，增减1day都只计算24小时
#### DataOffset相对时间差指，无论一天是23\24\25小时，增减1day都与当天相同的时间保持一致
#### 例如，英国当地时间 2020年03月29日，01:00:00 时钟向前调整 1 小时 变为 2020年03月29日，02:00:00，开始夏令时

In [19]:
ts = pd.Timestamp('2020-3-29 01:00:00', tz='Europe/Helsinki')
ts + pd.Timedelta(days=1)

Timestamp('2020-03-30 02:00:00+0300', tz='Europe/Helsinki')

In [20]:
ts + pd.DateOffset(days=1)

Timestamp('2020-03-30 01:00:00+0300', tz='Europe/Helsinki')

#### 这似乎有些令人头大，但只要把tz（time zone）去除就可以不用管它了，两者保持一致，除非要使用到时区变换

In [21]:
ts = pd.Timestamp('2020-3-29 01:00:00')
ts + pd.Timedelta(days=1)

Timestamp('2020-03-30 01:00:00')

In [22]:
ts + pd.DateOffset(days=1)

Timestamp('2020-03-30 01:00:00')

#### （b）增减一段时间
#### DateOffset的可选参数包括years/months/weeks/days/hours/minutes/seconds

In [23]:
pd.Timestamp('2020-01-01') + pd.DateOffset(minutes=20) - pd.DateOffset(weeks=2)

Timestamp('2019-12-18 00:20:00')

#### （c）各类offset对象
#### 如果要需要了解其他的，请点击[这里](https://pandas.pydata.org/pandas-docs/version/1.0.0/user_guide/timeseries.html#offset-aliases)

freq | D/B | W | (B)M/(B)Q/(B)Y | (B)MS/(B)QS/(B)YS | H | T | S | C |
:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
offset | DateOffset/BDay | Week | (B)MonthEnd/(B)QuarterEnd/(B)YearEnd | (B)MonthBegin/(B)QuarterBegin/(B)YearBegin | Hour | Minute | Second | CDay(定制工作日)

In [24]:
pd.Timestamp('2020-01-01') + pd.offsets.Week(2)

Timestamp('2020-01-15 00:00:00')

In [25]:
pd.Timestamp('2020-01-01') + pd.offsets.BQuarterBegin(1)

Timestamp('2020-03-02 00:00:00')

#### （d）序列的offset操作
#### 利用apply函数

In [26]:
pd.Series(pd.offsets.BYearBegin(3).apply(i) for i in pd.date_range('20200101',periods=3,freq='Y'))

0   2023-01-02
1   2024-01-01
2   2025-01-01
dtype: datetime64[ns]

#### 直接使用对象加减（部分会报出性能警告，可能在将来会优化）

In [27]:
pd.date_range('20200101',periods=3,freq='Y') + pd.offsets.BYearBegin(3)

DatetimeIndex(['2023-01-02', '2024-01-01', '2025-01-01'], dtype='datetime64[ns]', freq='A-DEC')

#### 定制offset，可以指定weekmask和holidays参数（思考为什么三个都是一个值）

In [28]:
pd.Series(pd.offsets.CDay(3,weekmask='Wed Fri',holidays='2020010').apply(i)
                                  for i in pd.date_range('20200105',periods=3,freq='D'))

0   2020-01-15
1   2020-01-15
2   2020-01-15
dtype: datetime64[ns]

## 二、时序的索引及属性
### 1. 索引切片
#### 这一部分几乎与第二章的规则完全一致

In [29]:
rng = pd.date_range('2020','2021', freq='W')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts.head()

2020-01-05    1.724458
2020-01-12    0.734404
2020-01-19   -1.281356
2020-01-26   -1.851091
2020-02-02   -0.631711
Freq: W-SUN, dtype: float64

In [30]:
ts['2020-01-26']

-1.8510912739860979

#### 合法字符自动转换为时间点

In [31]:
ts['2020-01-26':'20200726'].head()

2020-01-26   -1.851091
2020-02-02   -0.631711
2020-02-09   -0.527990
2020-02-16   -0.480961
2020-02-23    0.941153
Freq: W-SUN, dtype: float64

### 2. 子集索引

In [32]:
ts['2020-7'].head()

2020-07-05    0.111562
2020-07-12   -0.455323
2020-07-19    0.733693
2020-07-26    1.559166
Freq: W-SUN, dtype: float64

#### 支持混合形态索引

In [33]:
ts['2011-1':'20200726'].head()

2020-01-05    1.724458
2020-01-12    0.734404
2020-01-19   -1.281356
2020-01-26   -1.851091
2020-02-02   -0.631711
Freq: W-SUN, dtype: float64

### 3. 时间点的属性

#### 采用dt对象可以轻松获得关于时间的信息

In [34]:
pd.Series(ts.index).dt.week.head()

0    1
1    2
2    3
3    4
4    5
dtype: int64

In [35]:
pd.Series(ts.index).dt.day.head()

0     5
1    12
2    19
3    26
4     2
dtype: int64

#### 利用strftime可重新修改时间格式

In [36]:
pd.Series(ts.index).dt.strftime('%Y-间隔1-%m-间隔2-%d').head()

0    2020-间隔1-01-间隔2-05
1    2020-间隔1-01-间隔2-12
2    2020-间隔1-01-间隔2-19
3    2020-间隔1-01-间隔2-26
4    2020-间隔1-02-间隔2-02
dtype: object

#### 对于datetime对象可以直接通过属性获取信息

In [37]:
pd.date_range('2020','2021', freq='W').month

Int64Index([ 1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,  4,
             5,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,  8,  8,  8,  8,
             8,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12,
            12],
           dtype='int64')

In [38]:
pd.date_range('2020','2021', freq='W').weekday

Int64Index([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
            6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
            6, 6, 6, 6, 6, 6, 6, 6],
           dtype='int64')

## 三、重采样

#### 所谓重采样，就是指resample函数，它可以看做时序版本的groupby函数

### 1. resample对象的基本操作
#### 采样频率一般设置为[offset字符](https://pandas.pydata.org/pandas-docs/version/1.0.0/user_guide/timeseries.html#offset-aliases)

In [39]:
df_r = pd.DataFrame(np.random.randn(1000, 3),index=pd.date_range('1/1/2020', freq='S', periods=1000),
                  columns=['A', 'B', 'C'])

In [40]:
r = df_r.resample('3min')
r

In [41]:
r.sum()

,A,B,C
2020-01-01 00:00:00,15.559537,0.062902,0.641736
2020-01-01 00:03:00,-3.967424,18.580301,-6.379570
2020-01-01 00:06:00,13.022272,-11.281240,0.641661
2020-01-01 00:09:00,1.422444,7.791046,-9.274273
2020-01-01 00:12:00,34.982845,-6.594200,-8.728343
2020-01-01 00:15:00,1.122831,7.143961,7.204476


In [42]:
df_r2 = pd.DataFrame(np.random.randn(200, 3),index=pd.date_range('1/1/2020', freq='D', periods=200),
                  columns=['A', 'B', 'C'])
r = df_r2.resample('CBMS')
r.sum()

,A,B,C
2020-01-01,-1.917304,4.382481,4.126383
2020-02-03,1.892205,4.436343,0.397829
2020-03-02,-1.156596,-2.259613,1.834812
2020-04-01,1.239826,2.579936,-6.967821
2020-05-01,-9.136122,2.576332,-11.141305
2020-06-01,0.996610,-7.694855,-6.119147
2020-07-01,-5.452416,1.749533,2.098522


### 2. 采样聚合

In [43]:
r = df_r.resample('3T')

In [44]:
r['A'].mean()

2020-01-01 00:00:00    0.086442
2020-01-01 00:03:00   -0.022041
2020-01-01 00:06:00    0.072346
2020-01-01 00:09:00    0.007902
2020-01-01 00:12:00    0.194349
2020-01-01 00:15:00    0.011228
Freq: 3T, Name: A, dtype: float64

In [45]:
r['A'].agg([np.sum, np.mean, np.std])

,sum,mean,std
2020-01-01 00:00:00,15.559537,0.086442,0.998925
2020-01-01 00:03:00,-3.967424,-0.022041,0.983131
2020-01-01 00:06:00,13.022272,0.072346,1.040731
2020-01-01 00:09:00,1.422444,0.007902,1.064689
2020-01-01 00:12:00,34.982845,0.194349,0.963200
2020-01-01 00:15:00,1.122831,0.011228,0.950309


#### 类似地，可以使用函数/lambda表达式

In [46]:
r.agg({'A': np.sum,'B': lambda x: max(x)-min(x)})

,A,B
2020-01-01 00:00:00,15.559537,5.855395
2020-01-01 00:03:00,-3.967424,4.796084
2020-01-01 00:06:00,13.022272,6.300903
2020-01-01 00:09:00,1.422444,4.898869
2020-01-01 00:12:00,34.982845,6.474434
2020-01-01 00:15:00,1.122831,4.809297


### 3. 采样组的迭代
#### 采样组的迭代和groupby迭代完全类似，对于每一个组都可以分别做相应操作

In [47]:
small = pd.Series(range(6),index=pd.to_datetime(['2020-01-01 00:00:00', '2020-01-01 00:30:00'
                                                 , '2020-01-01 00:31:00','2020-01-01 01:00:00'
                                                 ,'2020-01-01 03:00:00','2020-01-01 03:05:00']))
resampled = small.resample('H')
for name, group in resampled:
    print("Group: ", name)
    print("-" * 27)
    print(group, end="\n\n")

Group:  2020-01-01 00:00:00
---------------------------
2020-01-01 00:00:00    0
2020-01-01 00:30:00    1
2020-01-01 00:31:00    2
dtype: int64

Group:  2020-01-01 01:00:00
---------------------------
2020-01-01 01:00:00    3
dtype: int64

Group:  2020-01-01 02:00:00
---------------------------
Series([], dtype: int64)

Group:  2020-01-01 03:00:00
---------------------------
2020-01-01 03:00:00    4
2020-01-01 03:05:00    5
dtype: int64



## 四、问题与练习
#### 【问题一】 如何对date_range进行批量加帧操作或对某一时间段加大时间戳密度？
#### 【问题二】 如何批量增加TimeStamp的精度？
#### 【问题三】 对于超出处理时间的时间点，是否真的完全没有处理方法？
#### 【问题四】 阅读完[这里](https://pandas.pydata.org/pandas-docs/version/1.0.0/user_guide/timedeltas.html)关于Timedelta的内容后，请谈谈为什么Pandas要引入两种时间差（Timedelta和DateOffset）？

#### 【练习一】 现有一份关于时间序列的虚拟数据，请解决以下问题：
#### （a）将时间列合并后作为索引。
#### （b）对于col1为0或1的行，时间索引加上工作日型的三个月。
#### （c）按照20年的间隔分类，对col2中的列进行聚合，找出组内非零整数的比例。

In [48]:
pd.read_csv('data/Time_series_one.csv').head()

,年,月,日,时,分,秒,col1,col2
0,1930,4,15,17,43,6,1,-2
1,1944,11,11,21,46,14,0,-2
2,1954,11,10,15,1,22,0,-1
3,1953,4,6,22,11,20,1,2
4,1956,3,3,17,5,23,0,-5
